## Secret Sharing

In this notebook, you'll learn some basic building blocks of Secure Multiparty Computation (MPC or SMPC). In doing so, you'll be able to more clearly reason about use cases where MPC is a good fit. Remember: the goal isn't to roll your own crypto, but instead to develop a basic mental model to understand why, where and how MPC can help.

If this is interesting to you and you'd like to learn more, please have a read through [Morten Dahl's informative blog series on this topic and more in MPC!](https://mortendahl.github.io/2017/06/04/secret-sharing-part1/) 

In [3]:
from random import randrange

In [4]:
x = 45

Let's try to "hide" our number in a few ways naively...

In [5]:
keys = [100, 22, 43, 56]

In [6]:
enc_x = x - sum(keys)

In [7]:
enc_x

-176

In [8]:
enc_x = x - sum(keys)

This "works", but it actually leaks information! Let's take another x as an example.

In [9]:
x = -1000

In [10]:
enc_x = x - sum(keys)

In [11]:
enc_x

-1221

Here, if someone was using the same keys, I could infer that one number is smaller than the other number or otherwise leak information about the original values.

We could naively continue to find mechanisms that make this more secure, but there is actually a better way if we know some cryptography-math. Let's look at how we can hide information better in a field.

In [12]:
Q = 431
x = 45

In [13]:
enc_x = (x - sum(keys)) % Q

In [14]:
enc_x

255

In [15]:
keys = [33, 52, 167, 55, 77]

In [16]:
enc_x = (x - sum(keys)) % Q

In [17]:
enc_x

92

Because a field helps the numbers "wrap", it hides the extra information that we leaked without it. 

![Cats and Clocks](https://c.tenor.com/yz_7VcX0WjYAAAAd/cat-changing-the-clock-changing-the-time.gif "A black cat is standing up and playing with the minute arm of a clock, tapping it forward playfully.")


### But Math?
Currently this "encryption scheme" is not very helpful, because I can't do math with it... if I just wanted to encrypt and decrypt, there are a bunch of schemes I could use. We are here to do math!

To make it easier, I just wanted to support addition. I could combine the secret with my "keys" so that if you add them all together, they equal the secret.

In [18]:
Q = 431
num_players = 5
x = 45

shares = [randrange(Q) for _ in range(num_players-1)]
shares += [(x - sum(shares)) % Q]


In [19]:
shares

[151, 262, 162, 200, 132]

In [44]:
sum(shares)

807

In [20]:
sum(shares) % Q

45

## Now let's try a negative number!

We can start the same way as our sum, but think about modular math. What happens when you "subtract" time (i.e. come 13 hours before noon.)

In [29]:
(12 - 13) + 24

23

To keep our "negative" in our positive field space, we add our Q back... now let's see how it works with our current scheme....

In [33]:
x = -55

shares = [randrange(Q) for _ in range(num_players-1)]
shares += [(x - sum(shares)) % Q]

shares

[93, 159, 31, 340, 184]

In [34]:
# now, try to "decrypt" your secret! Hint: take a look at the "addition result" and your field....

In [ ]:
# %load solutions/subtraction.py


Let's make this easier by creating some helper functions to build our shares and combine them to decrypt.

In [45]:
def create_additive_shares(secret, Q=431, num_players=5):
    shares = [randrange(Q) for _ in range(num_players-1)]
    shares += [(secret - sum(shares)) % Q]
    return shares

def decrypt(result, Q=431):
    return result if result <= Q/2 else result - Q

In [46]:
shares = create_additive_shares(-33)
decrypt(sum(shares) % Q)

-33

## Replicated Secret Sharing

What if we wanted to multiply instead of add?

In [47]:
a = 44
b = 55
Q=2147487

In [48]:
a_shares = create_additive_shares(a, Q=2147487, num_players=3)
b_shares = create_additive_shares(b, Q=2147487, num_players=3)

These are then distributed to three players: each gets two shares from of each secret.

Player 1 (owns A): keeps a1, a2, receives b2, b3
Player 2 (owns B): keeps b1, b2, receives a1, a3
Player 3: receives a2, a3, b1, b3


c1= a1b2 + a1b3 + a2b2 

c2= a1b1 + a3b1 + a3b2

c3= a2b1 + a3b3 + a2b3


c1 + c2 + c3 = a1(b1 + b2 + b3) + a2(b1 + b2 + b3) + a3(b1 + b2 + b3) 

             = (a1 + a2 + a3)(b1 + b2 + b3) 
             
             = ab

In [27]:
a1, a2, a3 = a_shares
b1, b2, b3 = b_shares

In [1]:
# write a method to multiply the shares. It should take the first share and the second share. They need to be
# multiplied together and stay in the field!

In [ ]:
# %load solutions/multiply.py


In [29]:
c1 = multiply(a1, b2) + multiply(a1, b3) + multiply(a2, b2)
c1 % Q

887134

In [30]:
c2= multiply(a1, b1) + multiply(a3, b1) + multiply(a3, b2)
c2 % Q

296786

In [31]:
c3= multiply(a2, b1) + multiply(a3, b3) + multiply(a2, b3)
c3 % Q

965987

In [32]:
(c1+c2+c3) % Q

2420

In [33]:
a*b

2420

We can see here though that if two of the players collude, they have enough to recover extra information about the secret. They might also use the final step to learn more about the other shares. To avoid this, each player can "blind" their intermediary value before sharing it and then reveal the blind to another party later.

Player 1 has c1, adds blind (r1), receives c2+r2, keeps r1

Player 2 has c2, adds blind (r2), receives c3+r3, keeps r2

Player 3 has c3, adds blind (r3), receives c1+r1, keeps r3

Player 1 calculates: c2+r2-r1

Player 2 calculates: c3+r3-r2

Player 3 calculates: c1+r1-r3

Combined, they get c1+c2+c3 without leaking the extra information.

## Challenge

- Can you write a way to securely generate the blinds and pass them among the participants?